In [17]:
# imports

import numpy as np 
import os
import csv 

import pandas as pd 
import matplotlib.pyplot as plt 

from filterpy.kalman import predict
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import KalmanFilter

from openpyxl import load_workbook

In [18]:
# settings

start_time = pd.Timestamp('2024-09-17 00:00:00-0400')
end_time = pd.Timestamp('2024-09-18 00:00:00-0400')

home_dir = "../../"

In [19]:
# extract energy burned data

# energy burned inverse to hunger
# maybe energy burned should be offset

df = pd.read_csv(home_dir + "PhysicalEffort.csv")
df['start'] = pd.to_datetime(df['start'])
df.set_index('start', inplace=True)
df.sort_index(inplace=True)

common_time = pd.date_range(start=start_time, end=end_time, freq='min')
effort_aligned = df['value'].reindex(common_time, method='nearest')
# effort_interpolated = df.interpolate(method = 'linear')
aligned_df = pd.DataFrame({
    'value': effort_aligned
})

processed_effort = aligned_df.to_numpy().flatten()

# for i, row in aligned_df.iterrows(): print(f"time: {i}, value: {row['value']}")

In [20]:
# extract physical effort data

# physical effort should be inverse to hunger (not necessarily)

df = pd.read_csv(home_dir + "BasalEnergyBurned.csv")
df['start'] = pd.to_datetime(df['start'])
df.set_index('start', inplace=True)
df.sort_index(inplace=True)
df = df[~df.index.duplicated(keep='first')]

common_time = pd.date_range(start=start_time, end=end_time, freq='min')
energy_aligned = df['value'].reindex(common_time, method='nearest')
aligned_df = pd.DataFrame({
    'value': energy_aligned
})

processed_energy = aligned_df.to_numpy().flatten()

# for i,row in aligned_df.iterrows(): print(f"time: {i}, value: {row['value']}")

In [21]:
# extract hunger times data



In [22]:
# P matrix

P = np.cov(np.array([processed_effort, processed_energy, ]))
P

array([[  2.60068004,  -4.07262065],
       [ -4.07262065, 238.36686387]])

In [23]:
# zs matrix

n_steps = len(processed_energy)
zs = np.column_stack((processed_effort, processed_energy, ))

In [ ]:
# initial state X

X = np.array([
    [0],
    [0],
])

In [ ]:
# process model matrix F

dt = 1  # 1 second time step

F = np.array([
    [1, 0],
    [0, 1],
])

In [ ]:
# measurement noise covariance matrix R

R = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

In [ ]:
# process noise covariance matrix Q

Q = Q_discrete_white_noise(dim=2, dt=1., var=1.0)

In [ ]:
# measurement matrix H

H = np.array([
    [0, -1, 0],  # energy mapping
    [0, 0, -1],  # effort mapping
])

In [ ]:
# kalman filter initialization

kf = KalmanFilter(dim_x=3, dim_z=2)
kf.x = X
kf.P = P
kf.R = R
kf.Q = Q
kf.F = F
kf.H = H